In [1]:
import numpy as np
import scipy.stats as sps
from glob import glob
import matplotlib.pyplot as plt
import ROOT as rt
import root_numpy as rtnp
import sys, os, yaml
sys.path.append('/storage/user/ocerri/BPhysics/lib')
from histo_utilities import create_TH1D, create_TH2D, std_color_list, SetMaxToMaxHist, make_ratio_plot
from cebefo_style import Set_2D_colz_graphics
from progressBar import ProgressBar

from analysis_utilities import drawOnCMSCanvas, getEff

import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 1
CMS_lumi.extraText = "     Internal"

donotdelete = []

Welcome to JupyROOT 6.12/07


In [2]:
# load FWLite C++ libraries
rt.gSystem.Load("libFWCoreFWLite.so");
rt.gSystem.Load("libDataFormatsFWLite.so");
rt.FWLiteEnabler.enable()

# load FWlite python libraries
from DataFormats.FWLite import Events
from DataFormats.FWLite import Handle

In [3]:
fname = '../BPhysics/data/cmsMC_private/'
fname += 'BP_Tag_antiB0_DstmHc_Hardbbbar_evtgen_ISGW2_PUc0_10-2-3/test_GEN-SIM.root'

In [4]:
handle = {}
handle['genP'] = [Handle('std::vector<reco::GenParticle>'), 'genParticles']

In [5]:
nB0 = []
for ie, event in enumerate(Events(fname)):
    print event.eventAuxiliary().event()
    prods = {}
    for k,v in handle.iteritems():
        event.getByLabel(v[1], v[0])
        prods[k] = v[0].product()
    
    evNB0 = 0
    for ip, p in enumerate(prods['genP']):
        if not abs(p.pdgId()) in [511, 521]:
            continue
        if p.numberOfDaughters() == 1:
            continue
        if p.pdgId() == 511:
            evNB0 +=1
        ln = str(p.pdgId()) + ' -->'
        for d in p.daughterRefVector():
            ln += ' ' + str(d.pdgId())
        print ip, ':', '({}-->)'.format(p.mother().pdgId()), ln
    nB0.append(evNB0)
    print ' '
nB0 = np.array(nB0)

2411
474 : (5-->) -521 --> 421 -211 111 113
960 : (511-->) -511 --> -413 411 -311
 
3856
1312 : (1-->) 521 --> -423 113 223 211 223 223
2135 : (-513-->) -511 --> -413 413 -311
 
4382
1355 : (-523-->) -521 --> 423 13 -14
1407 : (511-->) -511 --> -413 413 -311
 
9584
546 : (5-->) -511 --> -413 413 -311
 
12745
1233 : (513-->) 511 --> 311 -411 413 111
1474 : (-513-->) -511 --> -413 413 -311
 
13855
552 : (5-->) -511 --> -413 413 -311
777 : (513-->) 511 --> -413 211 -211 323 -311
 
14694
701 : (5-->) -511 --> -413 411 -311
 
15232
496 : (513-->) 511 --> -413 -13 14 22
509 : (-511-->) 511 --> 411 311 -413
 
19768
809 : (2-->) 511 --> -413 -13 14
1023 : (-513-->) -511 --> -413 411 -311
 
19676
1519 : (511-->) -511 --> -413 413 -311
 


In [6]:
print nB0

[0 0 0 0 1 1 0 2 1 0]


In [7]:
getEff(np.sum(nB0>1), nB0.shape[0])

[0.1, 0.09486832980505139]